In [8]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix, graycoprops

In [9]:
def load_label_map(file_path: str):
    """
    Reads a file with labels and creates a dictionary that maps the name of each image file to its matching number.

    Args:
        file_path (str): The file path to the text
                        (ex: 'labels/LABELS_processed.txt).

    Returns:
        dict: A dictionary where the keys are the filenames (str) and the values are the labels (int)
    """
    labels_maps = {}

    try:
        with open(file_path, 'r') as f:
            for line in f:
                clean_line = line.strip()

                if clean_line:
                    parts = clean_line.split(' ', 1)

                    if len(parts) == 2:
                        label_str, file_name = parts
                        labels_maps[file_name] = int(label_str)

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found")
        return None

    return labels_maps

In [10]:
from typing import Literal

def lbp(image: np.ndarray,
       P : int = 8,
       R : int = 2,
       method : Literal['default', 'ror', 'uniform', 'nri_uniform', 'var'] = 'nri_uniform'):
    assert isinstance(image, np.ndarray) and len(image.shape) == 2
    desc = local_binary_pattern(image, P, R, method=method)

    n_bins = P * ( P - 1) + 3
    hist, _ = np.histogram(desc, density=True, bins=n_bins, range=(0, n_bins))

    return hist


In [15]:
from typing import List, Union

def glcm(image : np.ndarray,
        distances : Union[List[int], np.ndarray] = [1,3,5],
        angles : Union[List[float], np.ndarray]  = np.deg2rad([0,90,180,270])):
    assert isinstance(image, np.ndarray) and len(image.shape) == 2
    hists = graycomatrix(image, distances=distances, angles=angles, normed=True, symmetric=True)
    prop_names = ["contrast", "dissimilarity", "homogeneity", "ASM", "energy", "correlation"]
    props = np.array([graycoprops(hits, prop).flatten() for prop in prop_names]).flatten()

    return props
